In [1]:
from pylab import *
from sympy import *
from sympy.solvers import solve
import transforms3d


init_printing()

# Initialisation des fonctions lambdifier pour la simulation
## Lambdification des calculs de Cd et Cl

In [2]:
def compute_sigma(alpha, alpha_0, alpha_s, delta_s):       #  Comme c'est une fonction avec des conditions, on peut pas la lambdifier
    if alpha>=alpha_s+delta_s:
         sigma=0.0
    elif alpha>=alpha_s:
         sigma=0.5*(1.0+cos(pi*(alpha+alpha_0-alpha_s)/delta_s))
    elif alpha>=-alpha_s:
         sigma=1.0
    elif alpha>=-alpha_s-delta_s:
         sigma=0.5*(1.0+cos(pi*(alpha+alpha_0+alpha_s)/delta_s))
    else:
         sigma=0.0
    return(sigma)


# Initialisation des paramètres pour les équations symbolique
CL_1_sa = Symbol('C^{sa}_{L,1}',real=True)             # Coeff rechercher

CLsa = Symbol('C_L^{sa}',real=True)
CLfp = Symbol('C_L^{fp}',real=True)
C_L = Symbol('C_L',real=True)

CD_0_sa = Symbol('C^{sa}_{D,0}',real=True)             # Coeff rechercher
CD_1_sa = Symbol('C^{sa}_{D,1}',real=True)             # Coeff rechercher
CD_0_fp = Symbol('C^{fp}_{D,0}',real=True)             # Coeff rechercher
CD_1_fp = Symbol('C^{fp}_{D,1}',real=True)             # Coeff rechercher

CDsa = Symbol('C_D^{sa}',real=True)
CLfp = Symbol('C_D^{fp}',real=True)
C_D = Symbol('C_D',real=True)                       

k_0 = Symbol('k_0', real=True)                         # coeff rechercher
k_1 = Symbol('k_1', real=True)                         # coeff rechercher
k_2 = Symbol('k_2', real=True)                         # coeff rechercher

sigma = Symbol('sigma',real=True)                     # Coef de transistion small angle / flat plate

delta_0 = Symbol('delta_0',real=True)                 # Angle des ailerons
delta_s = Symbol('delta_s', real=True)                # Coeff rechercher : largeur du stall

alpha_0 = Symbol('alpha_0',real=True)                 # Coeff rechercher 
alpha_s = Symbol('alpha_s',real=True)                 # Coeff rechercher
alpha = Symbol('alpha',real=True)                     # angle d'attaque

# Calcule des fonctions 
CL_sa = 1/2 * CL_1_sa * sin(2*(alpha + alpha_0))
CD_sa = CD_0_sa + CD_1_sa * sin(alpha + alpha_0)*sin(alpha + alpha_0)

CL_fp = 1/2 * CD_1_fp * sin(2*(alpha + alpha_0))
CD_fp = CD_0_fp + CD_1_fp * sin(alpha + alpha_0)*sin(alpha + alpha_0)

C_L = CL_fp + sigma*(CL_sa - CL_fp) - k_2 * sin(delta_0)
C_D = CD_fp + sigma*(CD_sa - CD_fp)

## Lambdification des fonctions que nous allons utiliser en simulation.
CL_function = lambdify((CL_1_sa, CD_1_fp, sigma, alpha, alpha_0, delta_0, k_1, k_2), C_L)          
CD_function = lambdify((CD_0_fp, CD_0_sa, CD_1_sa, CD_1_fp, sigma, alpha, alpha_0, k_0), C_D)

## Lambdification des caluls des efforts aérodynamiques des ailes et des moteurs : 

In [3]:
# Paramètres pour les équations symboliques pour les efforts aérodynamiques des ailes et des moteurs
B_B       = Matrix([[1,0,0], [0,1,0], [0,0,1]])                              # Base dans le repère body

omega1, omega2, omega3 = symbols('\omega_1, \omega_2, \omega_3', real=True)
Omega     = Matrix([omega1, omega2, omega3])                                 # Vecteur de rotation
 
r00, r01, r02, r10, r11, r12, r20, r21, r22 = symbols('r_{00}, r_{01}, r_{02}, r_{10}, r_{11}, r_{12}, r_{20}, r_{21}, r_{22}', real=True)
R         = Matrix([[r00,r01,r02], [r10,r11, r12], [r20, r21, r22]])          # Matrice de rotation
R_eleron         = R * rot_axis1(45*180/np.pi)                   # Matrice de rotation
R_eleron2         = R * rot_axis1(-45*180/np.pi)                   # Matrice de rotation

Vb1,Vb2,Vb3=symbols('V_{b1} V_{b2} V_{b3}',real=True)
v_B        = Matrix([Vb1, Vb2, Vb3])                                          # Vitesse du corps (repère drone)

Vw1,Vw2,Vw3=symbols('V_{w1} V_{w2} V_{w3}',real=True)                         # Vitesse du vent dans le repère NED 
v_W        = Matrix([Vw1, Vw2, Vw3])

xcp, ycp, zcp = symbols('x_{cp}, y_{cp}, z_{cp}')
X_cp     = Matrix([xcp, ycp, zcp])                                            # Position du centre de poussé d'un corps dans le repère body

A         = Symbol('A', real=True)                                            # 0.5 * rho * S 

VxNED, VyNED , VzNED = symbols('V_{NED_x} V_{NED_y} V_{NED_z}') 
Air_speed_earth  = Matrix([VxNED, VyNED , VzNED])                             # air speed dans le repère NED

VxBody, VyBody, VzBody = symbols('V_{body_x} V_{body_y} V_{body_z}') 
Air_speed_body   = Matrix([VxBody, VyBody, VzBody])                           # air speed dans le repère body

omega,ct=symbols('\omega,C_t',real=True)                                      # vitesse de rotation des moteurs, et coefficient de poussé des helices
Cd, Cl, Ct, Cq=symbols('C_d,C_l,C_t,C_q',real=True)                           # vitesse de rotation des moteurs, et coefficient de poussé des helices

motor_axis_in_body_frame = Matrix([1,0,0])                                    # Axe des moteurs, ici placé en mode avion

ch = Symbol('c_h')

omega_rotor = symbols('\omega_{rotor}', real=True)                            # Vitesse de rotation des moteurs
spinning_sense = symbols('\epsilon_{spin}', int=True)                         # Sens de rotation d'un moteur [-1 ou 1]

frontward_B = B_B[:,0]
upward_B    = B_B[:,2]
crossward_B = B_B[:,1]

def GenDirectForceWing(Omega, cp, vB, vW, R, crossward_Body):

    Air_speed_earth = vB - vW 
    Air_speed_body  = (R.T* Air_speed_earth) - cp.cross(Omega)
    VelinLDPlane    = Air_speed_body - Air_speed_body.dot(crossward_Body.T) * crossward_Body
    
    dragDirection = -VelinLDPlane / VelinLDPlane.norm()  #if VelinLDPlane_norm > VelLim else Matrix([0,0,0])
    liftDirection = crossward_Body.cross(dragDirection) #if crossward_NED.norm() > VelLim else Matrix([0,0,0])

    return VelinLDPlane, dragDirection, liftDirection

def GenForceWing(A, VelinLDPlane, dragDirection, liftDirection, Cd, Cl, cp):
    D = A * VelinLDPlane.norm()**2 * dragDirection * Cd
    L = A * VelinLDPlane.norm()**2 * liftDirection * Cl

    F_wing = L+D 
    Torque_wing =  cp.cross(F_wing)

    return F_wing, Torque_wing

def Generate_Sum_Force_wing(A_list, Omega, cp_list, R_list, vB, vW, crossward_body):
    p = 0
    Sum_Force_Wing = Matrix([0,0,0])
    Sum_Torque_Wing =  Matrix([0,0,0])
    for i in cp_list:
        VelinLDPlane, dragDirection, liftDirection = GenDirectForceWing(Omega, i, vB, vW, R_list[p], crossward_body)
        F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd, Cl, i)
        
        Sum_Force_Wing  = Sum_Force_Wing + F_wing
        Sum_Torque_Wing = Sum_Torque_Wing + Torque_wing
        p+=1

    return Sum_Force_Wing, Sum_Torque_Wing

def GenForceMoteur(Omega, ct, cq, omega_rotor, cp, vB, vW, ch, R, motor_axis_in_body_frame, spinning_sense):
    ## Cette fonction calcule les effort produit par un rotor sur le drone en fonction de son sens de rotation et de sa localisation, les efforts sont donnés
    ## dans le repère body. l'axe des moteur est placé suivant l'axe x du drone (mode avion seulement)
    Air_speed_earth = vB - vW
    air_speed_in_rotor_frame = (R.T* Air_speed_earth) - cp.cross(Omega)
    Axial_speed = air_speed_in_rotor_frame.dot(motor_axis_in_body_frame)
    lat_speed = air_speed_in_rotor_frame - (Axial_speed * (motor_axis_in_body_frame))
        
    T = ct*omega_rotor**2
    H = ch * omega_rotor
    
    T_vec = T * motor_axis_in_body_frame - H * lat_speed
    
    torque = - omega_rotor * cq * lat_speed
    torque = - spinning_sense * cq * T * motor_axis_in_body_frame 
    torque_at_body_center = torque + cp.cross(T_vec.T)
        
    return T_vec, torque_at_body_center

def Generate_Sum_Force_Moteur(Omega, ct, cq, omega_rotor, cp_list, vB, vW, ch, R, motor_axis_in_body_frame_list, spinning_sense_list):
    p = 0
    Sum_Force_Rotor = Matrix([0,0,0])
    Sum_Torque_Rotor =  Matrix([0,0,0])
    for cp in cp_list:
        F_rotor, Q_rotor = GenForceMoteur(Omega, ct, cq, omega_rotor, cp, vB, vW, ch, R, motor_axis_in_body_frame_list, spinning_sense_list[p])        
        Sum_Force_Rotor  = Sum_Force_Rotor + F_rotor
        Sum_Torque_Rotor = Sum_Torque_Rotor + Q_rotor
        p+=1

    return Sum_Force_Rotor, Sum_Torque_Rotor

# Initilisation des paramètres pour la génération des équations des ailes 
cp_list = [Matrix([0,0.45,0]), Matrix([0,-0.45,0]), Matrix([-0.5,0.15,0]), Matrix([-0.5,-0.15,0]), Matrix([0,0,0])]
A_list = [0.4* 1.292 * 0.5,0.4* 1.292 * 0.5,0.1* 1.292 * 0.5,0.1* 1.292 * 0.5,0.5* 1.292 * 0.5] 
R_list = [R, R, R_eleron, R_eleron2, R]

# Initialisation pour les équations des rotors
cp_list_rotor = [Matrix([0.2,0.45,0]), Matrix([0.2,-0.45,0]), Matrix([-0.2,0.45,0]), Matrix([-0.2,-0.45,0])]
spinning_sense_list = [1,-1,1,-1]

# Génération des équations : 
Sum_F_wing, Sum_Q_wing = Generate_Sum_Force_wing(A_list, Omega, cp_list, R_list, v_B, v_W, crossward_B)
#Sum_F_wing.simplify()
print("simplify wing force OK")
#Sum_Q_wing.simplify()
print("simplify wing couple OK")
Sum_F_rotor, Sum_Q_rotor = Generate_Sum_Force_Moteur(Omega, Ct, Cq, omega_rotor, cp_list_rotor, v_B, v_W, ch, R, motor_axis_in_body_frame, spinning_sense_list)
Sum_F_rotor.simplify()
Sum_Q_rotor.simplify()

# Création des équations des efforts pour une aile et un seul rotor 
VelinLDPlane, dragDirection, liftDirection =  GenDirectForceWing(Omega, X_cp, v_B, v_W, R, crossward_B)
Force_wing, Torque_wing = GenForceWing(A, VelinLDPlane, dragDirection, liftDirection, Cd, Cl, X_cp)
                                       
Force_rotor, Torque_rotor = GenForceMoteur(Omega, Ct, Cq, omega_rotor, X_cp, v_B, v_W,ch, R, motor_axis_in_body_frame, spinning_sense)

# Lambdification des fonction
Force_wing_function = lambdify((A, Cd, Cl, v_B, v_W, Omega, X_cp, R), Force_wing, 'sympy')
Torque_wing_function = lambdify((A, Cd, Cl, v_B, v_W, Omega, X_cp, R), Torque_wing, 'sympy')

Force_rotor_function = lambdify((Ct, omega_rotor, v_B, v_W, R, Omega, X_cp), Force_rotor)
Torque_rotor_function = lambdify((Cq, Ct, omega_rotor, Omega, R, v_B, v_W, X_cp, spinning_sense), Torque_rotor)

## Lambdification des fonctions que nous allons utiliser en simulation 
# Sommes des efforts des surfaces portantes
Sum_F_wing_function = lambdify((Omega, R, v_B, v_W, Cd, Cl), Sum_F_wing, 'sympy')
Sum_T_wing_function = lambdify((Omega, R, v_B, v_W, Cd, Cl), Sum_Q_wing, 'sympy')

# Somme des efforts des moteurs
Sum_F_rotor_function = lambdify((Omega, R, v_B, v_W, Ct, Cq, ch, omega_rotor), Sum_F_rotor, 'numpy')
Sum_Q_rotor_function = lambdify((Omega, R, v_B, v_W, Ct, Cq, ch, omega_rotor), Sum_Q_rotor, 'numpy')

# Cl, Cd
CL_function = lambdify((alpha, sigma, alpha_0, delta_0, CL_1_sa, CD_1_fp,k_1, k_2), C_L)          
CD_function = lambdify((alpha, sigma, alpha_0, CD_0_fp, CD_0_sa, CD_1_sa, CD_1_fp, k_0), C_D)

simplify wing force OK
simplify wing couple OK


Test des fonctions

In [4]:
#R_test     = Matrix([[1,0,0],[0,1,0],[0,0,1]])
#print(Sum_F_wing_function(Matrix([0,0,0]), R_test, Matrix([1,0,0]), Matrix([0,0,0]), 0.8,0.2))    

## Définition de l'angle d'attaque 

In [5]:
def compute_alpha(dragDirection, liftDirection, frontward_Body, VelinLDPlane): 
    calpha= dragDirection.dot(frontward_Body)
    absalpha= -acos(calpha)
    signalpha = sign(liftDirection.dot(frontward_Body)) 
    if VelinLDPlane.norm()>1e-7 :
        alpha = signalpha*absalpha 
    else :
        alpha=0
    if abs(alpha)>0.5*np.pi:
        if alpha>0 :alpha=alpha-np.pi 
        else: alpha=alpha+np.pi
    return alpha

## Equation de la dynamique 

In [6]:
def Dynamique(sum_force_rotor, sum_force_wing, sum_torque_rotor, sum_torque_wing, m, J, Omega):
    Accel = (sum_force_rotor + sum_force_wing) / m
    OmegaPoint = (sum_torque_rotor + sum_torque_wing - Omega.cross((J * Omega)))
    OmegaPoint = Matrix([OmegaPoint[0] / J[0], OmegaPoint[1] / J[4], OmegaPoint[2] / J[8]])
    return Accel, OmegaPoint



# Simulation Complète sur un pas de temps

# Initialisation de la simulation!

In [7]:
#Initialisation des états 
X = Matrix([0,0,0])
V = Matrix([0,1,0])
Accel = Matrix([0,0,0])
q = Quaternion(1,0,0,0)                       # norme toujours = 1 
Omega = Matrix([0,0,0])
OmegaP = Matrix([0,0,0])             # Accélération angulaire
omega_rotor = 100      # Vitesse angulaire des rotors
delta = 0                            # Angle de control des aileron
R = q.to_rotation_matrix()    


# Paramètres phyisiques :
m = 1.5
J = diag(0.19,0.15,0.15)
alpha_0 = 0.0
alpha_s = 0.3391428111
delta_s = 30.0*np.pi/180 
v_W = Matrix([0,0,1])

Ch = 0.0001
Ct = 1e-4

# Initialisation des coefs aéro : 
k1 = 0.5
k2 = 0.5
k0 = 0.5
cd1fp = 3.8
cl1fp = cd1fp

cl1sa = 5.5
cd1sa = 3.03

cd0sa = 0.02
cd0fp = cd0sa
pprint(R)

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦


In [8]:
## Initialisation des différents dictionnaires des valeurs

In [9]:
Dict_etats = {"position" : X, "vitesse" : V, "acceleration" : Accel, "orientation" : q, "vitesse_angulaire" : Omega, "accel_angulaire" : OmegaP}
Dict_parametres = {"masse": m , "inertie": J, "alpha0" :alpha_0, "alpha_stall" : alpha_s, "largeur_stall" : delta_s, "wind" : v_W}
Dict_variables = {"cd0sa" : cd0sa, "cl1fp" : cl1fp, "cd1sa" : cd1sa, "cl1sa" : cl1sa, "cd1fp" : cd1fp, "coeff_drag_shift":k0, "coeff_lift_shift":k1, "coef_lift_gain":k2 }
Dict_commande = {"delta" : delta}

In [15]:
dt =0.02
from time import *
from threading import get_ident
N = 100
Id = pthread_getcpuclockid(get_ident())

moyT =0
for i in range(N):

    #Initialisation des états 
    X = Matrix([0,0,0])
    V = Matrix([0,1,0])
    Accel = Matrix([0,0,0])
    q = Quaternion(1,0,0,0)                       # norme toujours = 1 
    Omega = Matrix([0,0,0])
    OmegaP = Matrix([0,0,0])             # Accélération angulaire
    omega_rotor = 100      # Vitesse angulaire des rotors
    delta = 0                            # Angle de control des aileron
    R = q.to_rotation_matrix()    


    # Paramètres phyisiques :
    m = 1.5
    J = diag(0.19,0.15,0.15)
    alpha_0 = 0.0
    alpha_s = 0.3391428111
    delta_s = 30.0*np.pi/180 
    v_W = Matrix([0,0,1])

    Ch = 0.0001
    Ct = 1e-4

    # Initialisation des coefs aéro : 
    k1 = 0.5
    k2 = 0.5
    k0 = 0.5
    cd1fp = 3.8
    cl1fp = cd1fp

    cl1sa = 5.5
    cd1sa = 3.03

    cd0sa = 0.02
    cd0fp = cd0sa
    t1 = clock_gettime(Id)

    VelinLDPlane, dragDirection, liftDirection =  GenDirectForceWing(Omega, X_cp, V, v_W, R, crossward_B)
    alpha = compute_alpha(dragDirection, liftDirection, frontward_B, VelinLDPlane)
    sigma =  compute_sigma((float(alpha)), alpha_0, alpha_s, delta_s)

    Cl = CL_function(float(alpha), float(sigma), alpha_0, delta, cl1sa, cd1fp,k1, k2)
    Cd = CD_function(float(alpha), float(sigma), alpha_0, cd0fp, cd0sa, cd1sa, cd1fp, k0)
    
    Wing_Force = Sum_F_wing_function(Omega, R, V, v_W, Cd, Cl)
    Wing_Torque = Sum_T_wing_function(Omega, R, V, v_W, Cd, Cl)
    

    Rotor_Force = Sum_F_rotor_function(Omega, R, V, v_W, Ct, Cq, Ch, omega_rotor)
    Rotor_Torque = Sum_Q_rotor_function(Omega, R, V, v_W, Ct, Cq, Ch, omega_rotor)

    Accel_body, OmegaP_body = Dynamique(Rotor_Force, Wing_Force, Rotor_Torque, Wing_Torque, m, J, Omega)
    t2 =clock_gettime(Id)

    
    moyT += t2-t1
print(moyT/N)

0.07060524170999984


Acc = R * Accel_body
OmegaP = R * OmegaP_body    


Omega = Omega + OmegaP*dt
V = V + N(Acc)*dt

X = X + N(V)*dt

pprint(N(X))
pprint(N(OmegaP*dt))
Dict_etats['acceleration'] = Matrix(N(Acc))
Dict_etats['vitesse_angulaire'] = Matrix(N(Omega))
Dict_etats = {"position" : X, "vitesse" : V, "acceleration" :  Matrix(N(Acc)), "orientation" : q,\
              "vitesse_angulaire" : Matrix(N(Omega)), "accel_angulaire" : Matrix(N(OmegaP))}


In [11]:
N(Omega)

TypeError: 'int' object is not callable

In [ ]:
N(Wing_Torque)

In [ ]:
pprint(Rotor_Force)

In [ ]:
pprint(Rotor_Torque)

In [ ]:
N(Accel_body)

In [ ]:
N(OmegaP_body)

In [ ]:
 N(Acc)